In [373]:
from __future__ import absolute_import, division, print_function

# Import TensorFlow >= 1.10 and enable eager execution
import tensorflow as tf

tf.enable_eager_execution()

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import time

print(tf.__version__)

1.12.0


In [374]:
import unidecode
filepath = "/Users/pallavitaneja/Downloads/fra-eng/fra.txt"
accented_string = u'Málaga'
# accented_string is of type 'unicode'
unaccented_string = unidecode.unidecode(accented_string)
# unaccented_string contains 'Malaga'and is of type 'str'
print(unaccented_string)
lines = open(filepath, encoding='UTF-8').read().strip().split('\n')

Malaga


In [375]:
at = tf.constant(3) 
bt = tf.constant(4)
mult = tf.multiply(at, bt)
print ("The multiplication produces:::", mult)


The multiplication produces::: tf.Tensor(12, shape=(), dtype=int32)


In [35]:
pl = tf.placeholder(tf.float32, name="p") 
pi = tf.constant(3.) 
c = tf.add(pl, pi)
print (c)

RuntimeError: tf.placeholder() is not compatible with eager execution.

In [376]:
# Converts the unicode file to ascii
import unidecode
filepath = "/Users/pallavitaneja/Downloads/fra-eng/fra.txt"
def sent_preprocessing(w):
    start_tag = '<SOS>'
    end_tag = '<EOS>'
    w = unidecode.unidecode(w.lower().strip())
    #Regex for padding punctuation with white spaces
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    #print(w)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    # For Example: 
    #i'm ok .  i m ok .
    #j'ai perdu. j ai perdu .
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    #print("w "+ w)
    w = w.rstrip().strip()
    # adding a start and an end token to the sentence to indicate to start and stop predicting.
    w = start_tag + ' ' + w + ' ' + end_tag
    return w

In [377]:
# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [FRENCH, ENGLISH]
def generate_sent_pairs(filepath, num_examples):
    lines = open(filepath, encoding='UTF-8').read().strip().split('\n')
    sent_pairs = []
    for line in lines[:num_examples]:
        l = []
        for sent in line.split('\t'):
            l.append(sent_preprocessing(sent))
        sent_pairs.append(l)
#     print(sent_pairs)
    return sent_pairs


In [378]:
# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENGLISH, SPANISH]
def create_dataset(filepath, num_examples):
    lines = open(filepath, encoding='UTF-8').read().strip().split('\n')
    
    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]
    
    return word_pairs

In [379]:
class LanguageIndex():
    def __init__(self, lang):
        self.lang = lang
        self.word2idx = {}
        self.idx2word = {}
        self.vocab = set()
    
        self.create_index()
    
    def create_index(self):
        for phrase in self.lang:
            self.vocab.update(phrase.split(' '))
        self.vocab = sorted(self.vocab)
        self.word2idx['<pad>'] = 0
        for index, word in enumerate(self.vocab):
            self.word2idx[word] = index + 1
        for word, index in self.word2idx.items():
            self.idx2word[index] = word

In [380]:
create_dataset(filepath, 100)

Go.
go.
go.
Va !
va !
va !
Hi.
hi.
hi.
Salut !
salut !
salut !
Run!
run!
run!
Cours !
cours !
cours !
Run!
run!
run!
Courez !
courez !
courez !
Wow!
wow!
wow!
Ça alors !
ca alors !
ca alors !
Fire!
fire!
fire!
Au feu !
au feu !
au feu !
Help!
help!
help!
À l'aide !
a l'aide !
a l'aide !
Jump.
jump.
jump.
Saute.
saute.
saute.
Stop!
stop!
stop!
Ça suffit !
ca suffit !
ca suffit !
Stop!
stop!
stop!
Stop !
stop !
stop !
Stop!
stop!
stop!
Arrête-toi !
arrete-toi !
arrete-toi !
Wait!
wait!
wait!
Attends !
attends !
attends !
Wait!
wait!
wait!
Attendez !
attendez !
attendez !
Go on.
go on.
go on.
Poursuis.
poursuis.
poursuis.
Go on.
go on.
go on.
Continuez.
continuez.
continuez.
Go on.
go on.
go on.
Poursuivez.
poursuivez.
poursuivez.
Hello!
hello!
hello!
Bonjour !
bonjour !
bonjour !
Hello!
hello!
hello!
Salut !
salut !
salut !
I see.
i see.
i see.
Je comprends.
je comprends.
je comprends.
I try.
i try.
i try.
J'essaye.
j'essaye.
j'essaye.
I won!
i won!
i won!
J'ai gagné !
j'ai gagne !
j'ai 

[['<start> go . <end>', '<start> va ! <end>'],
 ['<start> hi . <end>', '<start> salut ! <end>'],
 ['<start> run ! <end>', '<start> cours ! <end>'],
 ['<start> run ! <end>', '<start> courez ! <end>'],
 ['<start> wow ! <end>', '<start> ca alors ! <end>'],
 ['<start> fire ! <end>', '<start> au feu ! <end>'],
 ['<start> help ! <end>', '<start> a l aide ! <end>'],
 ['<start> jump . <end>', '<start> saute . <end>'],
 ['<start> stop ! <end>', '<start> ca suffit ! <end>'],
 ['<start> stop ! <end>', '<start> stop ! <end>'],
 ['<start> stop ! <end>', '<start> arrete toi ! <end>'],
 ['<start> wait ! <end>', '<start> attends ! <end>'],
 ['<start> wait ! <end>', '<start> attendez ! <end>'],
 ['<start> go on . <end>', '<start> poursuis . <end>'],
 ['<start> go on . <end>', '<start> continuez . <end>'],
 ['<start> go on . <end>', '<start> poursuivez . <end>'],
 ['<start> hello ! <end>', '<start> bonjour ! <end>'],
 ['<start> hello ! <end>', '<start> salut ! <end>'],
 ['<start> i see . <end>', '<start

In [381]:
generate_sent_pairs(filepath,100)

[['<SOS> go . <EOS>', '<SOS> va ! <EOS>'],
 ['<SOS> hi . <EOS>', '<SOS> salut ! <EOS>'],
 ['<SOS> run ! <EOS>', '<SOS> cours ! <EOS>'],
 ['<SOS> run ! <EOS>', '<SOS> courez ! <EOS>'],
 ['<SOS> wow ! <EOS>', '<SOS> ca alors ! <EOS>'],
 ['<SOS> fire ! <EOS>', '<SOS> au feu ! <EOS>'],
 ['<SOS> help ! <EOS>', '<SOS> a l aide ! <EOS>'],
 ['<SOS> jump . <EOS>', '<SOS> saute . <EOS>'],
 ['<SOS> stop ! <EOS>', '<SOS> ca suffit ! <EOS>'],
 ['<SOS> stop ! <EOS>', '<SOS> stop ! <EOS>'],
 ['<SOS> stop ! <EOS>', '<SOS> arrete toi ! <EOS>'],
 ['<SOS> wait ! <EOS>', '<SOS> attends ! <EOS>'],
 ['<SOS> wait ! <EOS>', '<SOS> attendez ! <EOS>'],
 ['<SOS> go on . <EOS>', '<SOS> poursuis . <EOS>'],
 ['<SOS> go on . <EOS>', '<SOS> continuez . <EOS>'],
 ['<SOS> go on . <EOS>', '<SOS> poursuivez . <EOS>'],
 ['<SOS> hello ! <EOS>', '<SOS> bonjour ! <EOS>'],
 ['<SOS> hello ! <EOS>', '<SOS> salut ! <EOS>'],
 ['<SOS> i see . <EOS>', '<SOS> je comprends . <EOS>'],
 ['<SOS> i try . <EOS>', '<SOS> j essaye . <EOS>']

In [382]:
class Word_Index:
    def __init__ (self, inp_lang):
        self.inp_lang = inp_lang
        self.vocab = set()
        self.w_idx = {} #word to index
        self.idx_w = {} #index to word
        
        self.lang_index_transform()
    def lang_index_transform(self):
        for line in self.inp_lang:
            self.vocab.update(line.split(' '))
        self.vocab = sorted(self.vocab)
        self.w_idx['<pad>'] = 0
        for idx,w in enumerate(self.vocab):
            self.w_idx[w] = idx+1
        for w, idx in self.w_idx.items():
            self.idx_w[idx] = w
        

In [383]:
sent_pairs = generate_sent_pairs(filepath, num_of_examples)
# sent_pairs
vocab = set()
inp_lang = LanguageIndex(sp for en, sp in sent_pairs)
# print(inp_lang.word2idx)
inp_lanuage = Word_Index(sp for en, sp in sent_pairs)
print(inp_lanuage.w_idx)
for first, second in sent_pairs:
    
    enc_inp_lang = LanguageIndex(second)
#print(first)
# print(enc_inp_lang.word2idx)
for line in first:
#     print(line)
    vocab.update(line)
vocab = sorted(vocab)
# print(vocab)
for first, second in sent_pairs:
    dec_target_lang = Word_Index(first)

{'<pad>': 0, '!': 1, ',': 2, '.': 3, '<EOS>': 4, '<SOS>': 5, '?': 6, 'a': 7, 'aboient': 8, 'accord': 9, 'agreable': 10, 'ah': 11, 'ai': 12, 'aide': 13, 'aidez': 14, 'ailleurs': 15, 'aller': 16, 'allez': 17, 'allons': 18, 'alors': 19, 'amuse': 20, 'amusez': 21, 'ans': 22, 'appelez': 23, 'appelle': 24, 'appellez': 25, 'apres': 26, 'arrete': 27, 'as': 28, 'asseyez': 29, 'assieds': 30, 'attaque': 31, 'attaquez': 32, 'attendez': 33, 'attends': 34, 'attention': 35, 'attrape': 36, 'attrapez': 37, 'au': 38, 'aucun': 39, 'aucune': 40, 'aussi': 41, 'avance': 42, 'avancez': 43, 'avertis': 44, 'avez': 45, 'avis': 46, 'avons': 47, 'battues': 48, 'battus': 49, 'beau': 50, 'bibi': 51, 'bien': 52, 'bientot': 53, 'bienvenue': 54, 'bizarre': 55, 'bon': 56, 'bonjour': 57, 'bouge': 58, 'boulot': 59, 'bras': 60, 'c': 61, 'ca': 62, 'calin': 63, 'calme': 64, 'calmes': 65, 'calmez': 66, 'camp': 67, 'capte': 68, 'cas': 69, 'casse': 70, 'ce': 71, 'ceci': 72, 'certain': 73, 'chauve': 74, 'chez': 75, 'chiens': 76

In [254]:
sent_pairs = generate_sent_pairs(filepath, num_of_examples)
    #create language indexes to create tensors
for first, second in sent_pairs:
    enc_inp_lang = Word_Index(second)
for first, second in sent_pairs:
    dec_target_lang = Word_Index(first)
print(dec_target_lang.vocab)
enc_inp_lang_tensor = []
for lang1, lang2 in sent_pairs:
    tensor =[]
    for w in lang2.split(' '):
        tensor.append(enc_inp_lang.w_idx[w])
    enc_inp_lang_tensor.append(tensor)
    
dec_target_lang_tensor = []
for lang1, lang2 in sent_pairs:
    tensor =[]
    for w in lang1.split(' '):
        tensor.append(dec_target_lang.w_idx[w]) 
    dec_target_lang_tensor.append(tensor) 

['', '.', '<', '>', 'E', 'O', 'S', 'a', 'i', 'm', 'o', 'r', 's', 'y']


KeyError: '<SOS>'

In [384]:
def create_tensors(filepath, num_of_examples):
    sent_pairs = generate_sent_pairs(filepath, num_of_examples)
    #create language indexes to create tensors
    enc_inp_lang = Word_Index(second for first, second in sent_pairs)
    dec_target_lang = Word_Index(first for first, second in sent_pairs)
    #create tensors, find_max_len tensor to pad all the inputs
    enc_inp_lang_tensor = []
    for lang1, lang2 in sent_pairs:
        tensor =[]
        for w in lang2.split(' '):
            tensor.append(enc_inp_lang.w_idx[w])
        enc_inp_lang_tensor.append(tensor)
    print(enc_inp_lang_tensor)
    dec_target_lang_tensor = []
    for lang1, lang2 in sent_pairs:
        tensor =[]
        for w in lang1.split(' '):
             tensor.append(dec_target_lang.w_idx[w]) 
        dec_target_lang_tensor.append(tensor) 
    
    enc_inp_max_len = max(len(tensor) for tensor in enc_inp_lang_tensor)
        
    dec_target_max_len = max(len(tensor) for tensor in dec_target_lang_tensor)
    #padding input tensors at encoder and target sensors at decoder
    
    enc_inp_lang_tensor = tf.keras.preprocessing.sequence.pad_sequences(enc_inp_lang_tensor, enc_inp_max_len, padding = 'post')
    
    dec_target_lang_tensor = tf.keras.preprocessing.sequence.pad_sequences(dec_target_lang_tensor, dec_target_max_len, padding = 'post')
    
    return enc_inp_lang_tensor, dec_target_lang_tensor, enc_inp_lang, dec_target_lang , enc_inp_max_len, dec_target_max_len


In [396]:
num_of_examples = 500
 
enc_inp_lang_tensor, dec_target_lang_tensor, enc_inp_lang, dec_target_lang , enc_inp_max_len, dec_target_max_len = create_tensors(filepath, num_of_examples)

#train-test splits
train_inp, val_inp, train_targ, val_targ = train_test_split(enc_inp_lang_tensor, dec_target_lang_tensor, test_size = 0.2)
len(train_inp), len(train_targ), len(val_inp), len(val_targ)

[[5, 408, 1, 4], [5, 342, 1, 4], [5, 88, 1, 4], [5, 87, 1, 4], [5, 62, 19, 1, 4], [5, 38, 158, 1, 4], [5, 7, 203, 13, 1, 4], [5, 345, 3, 4], [5, 62, 371, 1, 4], [5, 369, 1, 4], [5, 27, 389, 1, 4], [5, 34, 1, 4], [5, 33, 1, 4], [5, 291, 3, 4], [5, 85, 3, 4], [5, 292, 3, 4], [5, 57, 1, 4], [5, 342, 1, 4], [5, 197, 79, 3, 4], [5, 196, 139, 3, 4], [5, 196, 12, 170, 1, 4], [5, 197, 203, 12, 126, 1, 4], [5, 259, 254, 1, 4], [5, 31, 1, 4], [5, 32, 1, 4], [5, 344, 1, 4], [5, 7, 418, 344, 1, 4], [5, 236, 1, 4], [5, 380, 380, 1, 4], [5, 217, 389, 3, 4], [5, 408, 2, 220, 3, 4], [5, 17, 423, 220, 3, 4], [5, 410, 423, 220, 3, 4], [5, 196, 12, 285, 1, 4], [5, 80, 1, 4], [5, 285, 6, 4], [5, 80, 6, 4], [5, 376, 28, 68, 6, 4], [5, 241, 3, 4], [5, 242, 3, 4], [5, 353, 240, 91, 386, 60, 1, 4], [5, 354, 240, 91, 417, 60, 1, 4], [5, 197, 372, 392, 3, 4], [5, 197, 372, 391, 3, 4], [5, 197, 341, 3, 4], [5, 197, 372, 275, 3, 4], [5, 197, 372, 276, 3, 4], [5, 196, 12, 283, 3, 4], [5, 196, 12, 22, 3, 4], [5, 19

(400, 400, 100, 100)

In [397]:
BUFFER_SIZE = len(train_inp)
BATCH_SIZE = 64
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(enc_inp_lang.w_idx)
vocab_tar_size = len(dec_target_lang.w_idx)

dataset = tf.data.Dataset.from_tensor_slices((train_inp, train_targ)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

<BatchDataset shapes: ((64, 11), (64, 6)), types: (tf.int32, tf.int32)>


In [398]:
def gru(units):
#   If you have a GPU, we recommend using CuDNNGRU(provides a 3x speedup than GRU)
#   the code automatically does that.
    if tf.test.is_gpu_available():
        return tf.keras.layers.CuDNNGRU(units, 
                                    return_sequences=True, 
                                    return_state=True, 
                                    recurrent_initializer='glorot_uniform')
    else:
        return tf.keras.layers.GRU(units, 
                               return_sequences=True, 
                               return_state=True, 
                               recurrent_activation='sigmoid', 
                               recurrent_initializer='glorot_uniform')

In [399]:
class EncoderRNN(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, encoder_units, batch_size):
        super(EncoderRNN, self).__init__()
        self.batch_size = batch_size
        self.encoder_units = encoder_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.encoder_units)
     #forcing child class to override parent's methods   
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)        
        return output, state
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_size, self.encoder_units))

In [400]:
class AttnDecoderRNN(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, decoder_units, batch_size):
        super(AttnDecoderRNN, self).__init__()
        self.batch_size = batch_size
        self.decoder_units = decoder_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.decoder_units)
        self.fc = tf.keras.layers.Dense(vocab_size)
        
        # used for attention
        self.W1 = tf.keras.layers.Dense(self.decoder_units)
        self.W2 = tf.keras.layers.Dense(self.decoder_units)
        self.V = tf.keras.layers.Dense(1)
     
    #forcing child class to override parent's methods
    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        
        # hidden shape == (batch_size, hidden size)
        # hidden_expand shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_expand = tf.expand_dims(hidden, 1)
        
        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying tanh(FC(EO) + FC(H)) to self.V
        score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_expand)))
        
        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
        
        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)
        print(context_vector)
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
        # passing the concatenated vector to the GRU
        output, state = self.gru(x)
        
        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
        
        return x, state, attention_weights
        
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_size, self.decoder_units))

In [401]:
optimizer = tf.train.AdamOptimizer()


def loss_function(real, pred):
    mask = 1 - np.equal(real, 0)
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
    return tf.reduce_mean(loss_)

In [402]:
encoder = EncoderRNN(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = AttnDecoderRNN(vocab_tar_size, embedding_dim, units, BATCH_SIZE)
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [403]:
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()
    
    hidden = encoder.initialize_hidden_state()
    total_loss = 0
    
    for (batch, (inp, targ)) in enumerate(dataset):
        loss = 0
        
        with tf.GradientTape() as tape:
            enc_output, enc_hidden = encoder(inp, hidden)
            
            dec_hidden = enc_hidden
            
            dec_input = tf.expand_dims([dec_target_lang.w_idx['<SOS>']] * BATCH_SIZE, 1)       
            
            # Teacher forcing - feeding the target as the next input
            for t in range(1, targ.shape[1]):
                # passing enc_output to the decoder
                predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
                
                loss += loss_function(targ[:, t], predictions)
                
                # using teacher forcing
                dec_input = tf.expand_dims(targ[:, t], 1)
        
        batch_loss = (loss / int(targ.shape[1]))
        
        total_loss += batch_loss
        
        variables = encoder.variables + decoder.variables
        
        gradients = tape.gradient(loss, variables)
        
        optimizer.apply_gradients(zip(gradients, variables))
        
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix)
    
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / N_BATCH))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

NotImplementedError: When subclassing the `Model` class, you should implement a `call` method.

In [ ]:
def evaluate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    attention_plot = np.zeros((max_length_targ, max_length_inp))
    
    sentence = sent_preprocessing(sentence)

    inputs = [enc_inp_lang.w_idx[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    
    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([dec_target_lang.w_idx['<SOS>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        
        # storing the attention weigths to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += dec_target_lang.idx_w[predicted_id] + ' '

        if dec_target_lang.idx_w[predicted_id] == '<EOS>':
            return result, sentence, attention_plot
        
        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot

In [ ]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')
    
    fontdict = {'fontsize': 14}
    
    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    plt.show()

In [ ]:
def translate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    result, sentence, attention_plot = evaluate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)
        
    print('Input: {}'.format(sentence))
    print('Predicted translation: {}'.format(result))
    
    attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
    plot_attention(attention_plot, sentence.split(' '), result.split(' '))

In [ ]:
translate(u'appelez', encoder, decoder, enc_inp_lang, dec_target_lang, enc_inp_max_len, dec_target_max_len)